#Trabalho de IA 2 - 8 puzzle

##O objetivo desse trabalho é resolver o 8 puzzle utilizando outras 2 formas de diferentes algortimos: A* e busca gulosa.

####Prof. Leandro Alvim
####Júlia da Silva Borges

#Algoritmo A*

In [ ]:
import random
import time
import tracemalloc
import heapq
from collections import deque

Importações necessárias

Função heuristica_Manhatan

In [ ]:
def heuristica_Manhatan(estado):
    objetivo = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1), 0: (2, 2)
    }


- *objetivo*: Dicionário que define a posição correta (linha, coluna) de cada número no estado objetivo.

Para cada célula (i, j) do tabuleiro:

Se não for o zero, calcula a distância de Manhattan até sua posição correta:

dist
Soma essas distâncias para todas as peças — isso é h(n).

In [ ]:
    distancia = 0
        for i in range(3):
            for j in range(3):
                valor = estado[i][j]
                if valor != 0:
                    objetivo_i, objetivo_j = objetivo[valor]
                    distancia += abs(i - objetivo_i) + abs(j - objetivo_j)
        return distancia

- Para cada célula (i, j) do tabuleiro:

 - Se não for o zero, calcula a distância de Manhattan até sua posição correta:

   **distancia += abs(i - objetivo_i) + abs(j - objetivo_j)**

- A soma essas distâncias para todas as peças é *h(n).*

Função A*

In [ ]:

def a_estrela(estado_inicial):
    prioridade = []
    heapq.heappush(prioridade, (heuristica_Manhatan(estado_inicial), 0, estado_inicial))

    estado_anterior = {}
    custo_real = {tuple(map(tuple, estado_inicial)): 0}
    estados_visitados = set()

    node_expanded = 0
    max_depth = 0
    max_fringe_size = 1



- prioridade: fila de prioridade com tuplas (f, g, estado)

  - f = g + h: custo total estimado

  - g = 0 no início, pois ainda não fizemos nenhum movimento

- heapq.heappush insere f, g e o estado inicial na fila *prioridade*

- estado_inicial: matriz 3x3 com os números

- estado_anterior: para reconstruir o caminho da solução.

- custo_real: custo real mínimo conhecido até cada estado.

  - Os estados são convertidos para tuplas de tuplas para poderem ser usados como chaves de dicionário.

- estados_visitados: conjunto para evitar reexplorar estados já expandidos.

- node_expanded = quantos nós foram expandidos
- max_depth =  profundidade máxima da árvore de busca
- max_fringe_size = maior número de nós simultaneamente na fila de prioridade

In [ ]:
    while prioridade:
                _, custo_atual, estado_atual = heapq.heappop(prioridade)
                estado_atual_tupla = tuple(map(tuple, estado_atual))

                if estado_atual_tupla in estados_visitados:
                    continue

                estados_visitados.add(estado_atual_tupla)
                node_expanded += 1

- Enquanto houver elementos na fila de prioridade, continua expandindo nós.

  - Remove o estado com menor f(n) da fila (prioridade).

  - _: ignora o valor f (heurística + custo real), pois não será usado aqui.
  - custo_atual: é o custo real g(n) para chegar até esse estado.
  - estado_atual: é o estado do tabuleiro (matriz 3x3) a ser expandido.
  - Converte a matriz (listas de listas) em uma tupla imutável.

  - Se o estado já foi visitado, ele é ignorado.
  - O estado_atual é registrado como "visitado"
  - Conta quantos nós (estados) foram processados ao todo.



In [ ]:
        if estado_esperado(estado_atual):
                return (caminho_total(estado_anterior, estado_atual), node_expanded, custo_atual, max_depth, len(prioridade), max_fringe_size)

            for vizinho in vizinhos(estado_atual):
                tupla_vizinho = tuple(map(tuple, vizinho))
                novo_custo = custo_real[estado_atual_tupla] + 1

- Se o estado atual for o final ([1,2,3],[4,5,6],[7,8,0]), a função retorna a solução:
  - O caminho até a solução.
  - Quantos nós foram expandidos.
  - A profundidade (custo).
  - Profundidade máxima atingida.
  - Tamanho da fila atual.
  - Tamanho máximo da fila ao longo do processo.

- Gera todos os estados vizinhos possíveis:
  - A função vizinhos retorna uma lista de estados possíveis com 1 movimento a partir do estado atual.
  - Converte a matriz para tupla, para poder usar em dicionários e sets.
  - Calcula novo custo: O custo real até o vizinho é 1 a mais que o do estado atual. (Cada movimento custa 1.)

In [ ]:
            if tupla_vizinho not in custo_real or novo_custo < custo_real[tupla_vizinho]:
                    custo_real[tupla_vizinho] = novo_custo
                    f = novo_custo + heuristica_Manhatan(vizinho)
                    heapq.heappush(prioridade, (f, novo_custo, vizinho))
                    estado_anterior[tupla_vizinho] = estado_atual_tupla
                    max_depth = max(max_depth, novo_custo)
                    max_fringe_size = max(max_fringe_size, len(prioridade))

        return None, node_expanded, 0, max_depth, 0, max_fringe_size

- Se for a primeira vez que esse vizinho aparece OU o novo caminho é mais curto do que o anterior:
  -  Então atualiza as informações dele.
  - Calcula f(n) = g(n) + h(n), e adiciona o vizinho ao heap.
  - Salva que o vizinho veio do estado atual (usado para reconstruir o caminho depois).
  - max_depth: a maior profundidade já alcançada durante a busca
  - max_fringe_size: maior tamanho que a fila (fringe) atingiu.

- Retorna os dados com a solução como None.

Função Estado esperado

In [ ]:
def estado_esperado(estado_atual):
    esperado = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    return estado_atual == esperado

- Essa função verifica se o estado atual do puzzle é o estado final desejado (meta).
  - Define o estado **esperado** do 8-puzzle.
  - Retorna True se o estado atual for exatamente igual ao objetivo.

Função Vizinhos

In [ ]:
def vizinhos(estado_atual):
    vizinho = []
    row, col = next((r, c) for r in range(3) for c in range(3) if estado_atual[r][c] == 0)

-  Inicializa a lista que vai guardar os estados vizinhos gerados.
- Faz um loop por todas as posições (r, c) e retorna a primeira onde estado_atual[r][c] == 0.

In [ ]:
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            novo_r, novo_c = row + dr, col + dc
            if 0 <= novo_r < 3 and 0 <= novo_c < 3:
                novo_estado = [linha[:] for linha in estado_atual]
                novo_estado[row][col], novo_estado[novo_r][novo_c] = novo_estado[novo_r][novo_c], novo_estado[row][col]
                vizinho.append(novo_estado)

    return vizinho

-  Define os 4 **movimentos possíveis**: cima, baixo, esquerda e direita.
  - Calcula a nova posição do 0 após o movimento.
  - Garante que a nova posição está **dentro dos limites do tabuleiro 3x3**.
    - Cria uma cópia do estado atual.
    - Troca a posição do `0` com o número na nova posição
    - Adiciona o novo estado gerado à lista de vizinhos.
- Retorna a lista com todos os **estados alcançáveis** a partir do estado atual com 1 movimento.

Função Caminho Total

In [ ]:
def caminho_total(estado_anterior, estado_atual):
    caminho = [estado_atual]
    while tuple(map(tuple, estado_atual)) in estado_anterior:
        estado_atual = [list(row) for row in estado_anterior[tuple(map(tuple, estado_atual))]]
        caminho.append(estado_atual)
    return caminho[::-1]

- Reconstrói o **caminho da solução**, do estado final até o inicial, com base no dicionário `estado_anterior`.
  - Começa com o estado final.
  - Enquanto esse estado tiver um “pai”, continua voltando.
    - Obtém o estado anterior (pai) e o transforma de volta para matriz.
    - Adiciona o estado ao caminho.
- Retorna o caminho construído de trás para frente.



Função É Solúvel

In [ ]:
def eh_soluvel(puzzle):
    flat_puzzle = [num for row in puzzle for num in row if num != 0]
    inversoes = 0
    for i in range(len(flat_puzzle)):
        for j in range(i + 1, len(flat_puzzle)):
            if flat_puzzle[i] > flat_puzzle[j]:
                inversoes += 1
    return inversoes % 2 == 0

- Transforma a matriz para uma lista linear e exclui o `0` (espaço vazio), pois ele **não conta** nas inversões.
- Contador de quantas inversões existem no puzzle.
- A regra do 8-puzzle é um puzzle só é solúvel se o número de inversões for par.
  - Se for par, retorna True, caso contrário, False.

Função Gerar 8 puzzle

In [ ]:
def gerar_8puzzle():
    while True:
        elementos = list(range(9))
        random.shuffle(elementos)
        puzzle = [elementos[i:i+3] for i in range(0, 9, 3)]
        if eh_soluvel(puzzle):
            print("Estado inicial gerado:")
            for linha in puzzle:
                print(linha)
            return puzzle

- Entra em um loop infinito até gerar um puzzle solúvel.
  - Cria uma lista com os números de 0 a 8.
  - Embaralha os elementos aleatoriamente.
  - Converte a lista em uma matriz 3x3.
  - Verifica se o puzzle gerado é solúvel
    - Imprime o estado inicial gerado.
  - Retorna o puzzle solúvel gerado.

In [ ]:
estado_inicial = gerar_8puzzle()
inicio = time.time()
solucao, nos_expandidos, search_depth, max_search_depth, fringe_size, max_fringe_size = a_estrela(estado_inicial)
fim = time.time()

- Gera um estado inicial solúvel.
- Marca o tempo de início da execução.
- Chama o algoritmo A* passando o estado inicial e retorna `solucao`, `nos_expandidos`, `search_depth`,`max_search_depth` , `fringe_size` e `max_fringe_size`.
- Marca o tempo final da execução.

In [ ]:
if solucao:
    print(f"\ncost_of_path: {len(solucao) - 1}")
    print("\nPasso a passo da solução:\n")
    for passo, estado in enumerate(solucao):
        print(f"Passo {passo}:")
        for linha in estado:
            print(" ".join(" " if x == 0 else str(x) for x in linha))
        print("--------")

    print(f"nodes_expanded: {nos_expandidos}")
    print(f"running_time: {fim - inicio:.4f}")
    print(f"search depth: {search_depth}")
    print(f"max search depth: {max_search_depth}")
    print(f"fringe_size: {fringe_size}")
    print(f"max_fringe_size: {max_fringe_size}")

else:
    print("Não foi possível encontrar uma solução.")

- Se uma solução foi encontrada:
  - Imprime o custo da solução.
  - Mostra cada passo da solução.
  - Mostra:
    - Quantos nós foram expandidos.
    - Tempo de execução.
    - Profundidade da solução.
    - Profundidade máxima atingida.
    - Tamanho atual e máximo da fila de prioridade (fringe).

In [ ]:
tracemalloc.start()
a_estrela(estado_inicial)
memoria_usada = tracemalloc.get_traced_memory()[1]
tracemalloc.stop()
print(f"max_ram_usage: {memoria_usada}")

- Inicia o monitoramento de uso de memória.
- Executa novamente o A* apenas para medir o uso de memória.
- Captura o pico de uso de memória (em bytes).
- Encerra o monitoramento de memória.
- Exibe o uso máximo de RAM durante a execução do A*.

#Algoritmo Busca Gulosa

In [ ]:
import random
import time
import tracemalloc
import heapq
from collections import deque


def heuristica(estado):
    objetivo = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1), 0: (2, 2)
    }
    distancia = 0
    for i in range(3):
        for j in range(3):
            valor = estado[i][j]
            if valor != 0:
                objetivo_i, objetivo_j = objetivo[valor]
                distancia += abs(i - objetivo_i) + abs(j - objetivo_j)
    return distancia


- Define a função heurística usada para estimar o custo até o estado final.
  - Define as posições corretas de cada número no puzzle resolvido.
  - Inicializa a distância total acumulada.
  - Percorre todas as posições (linha i, coluna j) da matriz do estado atual.
    - Pega o valor naquela posição e ignora o 0
      - Recupera a posição correta do valor com base no dicionário objetivo.
      - Soma a distância de Manhattan do valor à sua posição correta.

  - Retorna a soma total das distâncias.

Função Busca Gulosa

In [ ]:
def busca_gulosa(estado_inicial):
    objetivo = [[1,2,3],[4,5,6],[7,8,0]]
    objetivo_tuple = tuple(map(tuple, objetivo))

    fila = []
    heapq.heappush(fila, (heuristica(estado_inicial), estado_inicial))

    visitados = set()
    estado_anterior = {}
    profundidade = {tuple(map(tuple, estado_inicial)): 0}

    max_fringe_size = 1
    node_expanded = 0
    max_depth = 0


- Define o estado final desejado e o converte em tuplas para ser usado como chave em dicionários.
- Cria a fila de prioridade e insere o estado inicial com sua heurística como prioridade.
- visitados: guarda os estados já expandidos.
- estado_anterior: armazena os pais de cada estado para reconstruir o caminho.
- profundidade: profundidade de cada estado visitado (quantos passos desde o início).
- Inicializa:
  - max_fringe_size: maior tamanho que a fila atingiu.
  - node_expanded: nós expandidos.
  - max_depth: profundidade máxima atingida.



In [ ]:
    while fila:
        _, estado_atual = heapq.heappop(fila)
        node_expanded += 1

        estado_tupla = tuple(map(tuple, estado_atual))

        if estado_tupla == objetivo_tuple:
            return caminho_total(estado_anterior, estado_atual), node_expanded, profundidade[estado_tupla], max_depth, len(fila), max_fringe_size

        if estado_tupla in visitados:
            continue

        visitados.add(estado_tupla)
        profundidade_atual = profundidade[estado_tupla]

- Enquanto houver estados na fila:
  - Remove o estado com menor heurística da fila.
  - Incrementa o número de nós expandidos.
  - Converte o estado atual para tupla imutável para ser usado como chave.
  - Verifica se o estado atual é o estado objetivo:
    - Retorna: O caminho da solução, nós expandidos e a profundidade da solução.
  - Pula esse estado se ele já foi visitado antes.
  - Marca o estado como visitado e pega sua profundidade atual.


In [ ]:
        for vizinho in vizinhos(estado_atual):
            tupla_vizinho = tuple(map(tuple, vizinho))
            if tupla_vizinho not in visitados:
                estado_anterior[tupla_vizinho] = estado_tupla
                profundidade[tupla_vizinho] = profundidade_atual + 1
                prioridade = heuristica(vizinho)
                heapq.heappush(fila, (prioridade, vizinho))

                max_depth = max(max_depth, profundidade[tupla_vizinho])
                max_fringe_size = max(max_fringe_size, len(fila))

    return None, node_expanded, 0, max_depth, 0, max_fringe_size

- Para cada estado vizinho (movimento possível), verifica se ele ainda não foi visitado.
  - Salva quem gerou esse vizinho e atualiza sua profundidade.
  - Calcula a heurística do vizinho e o insere na fila com essa prioridade.
  - Atualiza o valor da profundidade máxima e tamanho máximo da fila.
- Se sair do laço sem achar solução, retorna que não foi possível resolver.

A partir daqui, as funções se repetem.

In [ ]:
def caminho_total(estado_anterior, estado_atual):
    caminho = [estado_atual]
    while tuple(map(tuple, estado_atual)) in estado_anterior:
        estado_atual = [list(row) for row in estado_anterior[tuple(map(tuple, estado_atual))]]
        caminho.append(estado_atual)
    return caminho[::-1]

def vizinhos(estado_atual):
    vizinho = []
    row, col = next((r, c) for r in range(3) for c in range(3) if estado_atual[r][c] == 0)
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        novo_r, novo_c = row + dr, col + dc
        if 0 <= novo_r < 3 and 0 <= novo_c < 3:
            novo_estado = [linha[:] for linha in estado_atual]
            novo_estado[row][col], novo_estado[novo_r][novo_c] = novo_estado[novo_r][novo_c], novo_estado[row][col]
            vizinho.append(novo_estado)
    return vizinho

def eh_soluvel(puzzle):
    flat_puzzle = [num for row in puzzle for num in row if num != 0]
    inversoes = 0
    for i in range(len(flat_puzzle)):
        for j in range(i + 1, len(flat_puzzle)):
            if flat_puzzle[i] > flat_puzzle[j]:
                inversoes += 1
    return inversoes % 2 == 0

def gerar_8puzzle():
    while True:
        elementos = list(range(9))
        random.shuffle(elementos)
        puzzle = [elementos[i:i+3] for i in range(0, 9, 3)]
        if eh_soluvel(puzzle):
            print("Estado inicial gerado:")
            for linha in puzzle:
                print(linha)
            return puzzle


estado_inicial = gerar_8puzzle()

inicio = time.time()
solucao, nos_expandidos, search_depth, max_search_depth, fringe_size, max_fringe_size = busca_gulosa(estado_inicial)
fim = time.time()

if solucao:
    print(f"\ncost_of_path: {len(solucao) - 1}")
    print("\nPasso a passo da solução:\n")
    for passo, estado in enumerate(solucao):
        print(f"Passo {passo}:")
        for linha in estado:
            print(" ".join(" " if x == 0 else str(x) for x in linha))
        print("--------")

    print(f"nodes_expanded: {nos_expandidos}")
    print(f"running_time: {fim - inicio:.4f}")
    print(f"search depth: {search_depth}")
    print(f"max search depth: {max_search_depth}")
    print(f"fringe_size: {fringe_size}")
    print(f"max_fringe_size: {max_fringe_size}")
else:
    print("Não foi possível encontrar uma solução.")

tracemalloc.start()
busca_gulosa(estado_inicial)
memoria_usada = tracemalloc.get_traced_memory()[1]
tracemalloc.stop()
print(f"max_ram_usage: {memoria_usada}")

#Relatório Final

## Tabela Comparativa dos 5 Algoritmos

| Algoritmo  | Custo Médio (Aprox.) | Nós Expandidos | Tempo (s) | Profundidade Média | RAM (B) | Solução ótima | Uso de Memória |
| ---------- | -------------------- | -------------- | --------- | ------------------ | ------- | ------------- | -------------- |
| **BFS**    | 21.67                | 90.185         | 2.80      | 21.67              | 37.2 MB | ✅ Sim         | Alta           |
| **DFS**    | 40.431               | 50.072         | 7.64      | 40.431             | 69.1 MB | ❌ Não         | Alta           |
| **IDFS**   | 29.67                | 29.927         | 22.91     | 29.67              | 24.4 MB | ✅ Sim         | Média          |
| **A\***    | \~18.33              | 1.384          | 0.088     | 18.33              | 1.3 MB  | ✅ Sim         | Baixa          |
| **Gulosa** | \~33                 | 75             | 0.0015    | 33                 | 64 KB   | ❌ Não         | Muito baixa    |



## Análise Comparativa

BFS continua sendo confiável para soluções ótimas, mas com alto custo de memória e tempo relativamente maior. É robusto, mas pesado.

DFS foi o pior algoritmo no geral: soluções profundas demais, alto custo, e ainda assim sem garantia de otimalidade.

IDFS manteve a otimalidade como o BFS, mas com menor uso de memória. No entanto, o tempo de execução foi o mais alto entre todos.

A* encontrou soluções ótimas, com poucos nós expandidos, tempo baixo, e memória muito bem gerida. É o melhor algoritmo, considerando eficiência e qualidade da solução.

Busca Gulosa foi o mais rápido e o que menos expandiu nós, mas não garante a melhor solução. Em problemas simples ou onde a qualidade da solução não é crítica, pode ser útil.


## Conclusão

O A* destacou-se como o algoritmo mais eficiente no geral: rápido, econômico em memória e sempre retornando a solução ótima. A busca gulosa foi ainda mais rápida, mas não garantiu otimalidade.

Em termos de eficiência:

Melhor geral: A*, pelo ótimo equilíbrio entre tempo, memória e qualidade.

Mais rápido: Busca Gulosa, mas sem garantia de qualidade.

Mais confiável, mas pesado: BFS.

Alternativa de baixo uso de memória: IDFS, embora lenta.

Pior desempenho geral: DFS, por falta de otimalidade e uso excessivo de profundidade e memória.

Assim, para aplicações práticas que exigem qualidade e eficiência, A* é a melhor escolha dentre os cinco algoritmos avaliados

#Código completo A*

In [ ]:
import random
import time
import tracemalloc
import heapq


def heuristica_Manhatan(estado):
    objetivo = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1), 0: (2, 2)
    }
    distancia = 0
    for i in range(3):
        for j in range(3):
            valor = estado[i][j]
            if valor != 0:
                objetivo_i, objetivo_j = objetivo[valor]
                distancia += abs(i - objetivo_i) + abs(j - objetivo_j)
    return distancia

def a_estrela(estado_inicial):
    prioridade = []
    heapq.heappush(prioridade, (heuristica_Manhatan(estado_inicial), 0, estado_inicial))

    estado_anterior = {}
    custo_real = {tuple(map(tuple, estado_inicial)): 0}
    estados_visitados = set()

    node_expanded = 0
    max_depth = 0
    max_fringe_size = 1

    while prioridade:
        _, custo_atual, estado_atual = heapq.heappop(prioridade)
        estado_atual_tupla = tuple(map(tuple, estado_atual))

        if estado_atual_tupla in estados_visitados:
            continue

        estados_visitados.add(estado_atual_tupla)
        node_expanded += 1

        if estado_esperado(estado_atual):
            return (caminho_total(estado_anterior, estado_atual), node_expanded, custo_atual, max_depth, len(prioridade), max_fringe_size)

        for vizinho in vizinhos(estado_atual):
            tupla_vizinho = tuple(map(tuple, vizinho))
            novo_custo = custo_real[estado_atual_tupla] + 1

            if tupla_vizinho not in custo_real or novo_custo < custo_real[tupla_vizinho]:
                custo_real[tupla_vizinho] = novo_custo
                f = novo_custo + heuristica_Manhatan(vizinho)
                heapq.heappush(prioridade, (f, novo_custo, vizinho))
                estado_anterior[tupla_vizinho] = estado_atual_tupla
                max_depth = max(max_depth, novo_custo)
                max_fringe_size = max(max_fringe_size, len(prioridade))

    return None, node_expanded, 0, max_depth, 0, max_fringe_size

def estado_esperado(estado_atual):
    esperado = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]
    return estado_atual == esperado

def vizinhos(estado_atual):
    vizinho = []
    row, col = next((r, c) for r in range(3) for c in range(3) if estado_atual[r][c] == 0)

    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        novo_r, novo_c = row + dr, col + dc
        if 0 <= novo_r < 3 and 0 <= novo_c < 3:
            novo_estado = [linha[:] for linha in estado_atual]
            novo_estado[row][col], novo_estado[novo_r][novo_c] = novo_estado[novo_r][novo_c], novo_estado[row][col]
            vizinho.append(novo_estado)

    return vizinho

def caminho_total(estado_anterior, estado_atual):
    caminho = [estado_atual]
    while tuple(map(tuple, estado_atual)) in estado_anterior:
        estado_atual = [list(row) for row in estado_anterior[tuple(map(tuple, estado_atual))]]
        caminho.append(estado_atual)
    return caminho[::-1]

def eh_soluvel(puzzle):
    flat_puzzle = [num for row in puzzle for num in row if num != 0]
    inversoes = 0
    for i in range(len(flat_puzzle)):
        for j in range(i + 1, len(flat_puzzle)):
            if flat_puzzle[i] > flat_puzzle[j]:
                inversoes += 1
    return inversoes % 2 == 0

def gerar_8puzzle():
    while True:
        elementos = list(range(9))
        random.shuffle(elementos)
        puzzle = [elementos[i:i+3] for i in range(0, 9, 3)]
        if eh_soluvel(puzzle):
            print("Estado inicial gerado:")
            for linha in puzzle:
                print(linha)
            return puzzle


estado_inicial = gerar_8puzzle()


inicio = time.time()
solucao, nos_expandidos, search_depth, max_search_depth, fringe_size, max_fringe_size = a_estrela(estado_inicial)
fim = time.time()

if solucao:
    print(f"\ncost_of_path: {len(solucao) - 1}")
    print("\nPasso a passo da solução:\n")
    for passo, estado in enumerate(solucao):
        print(f"Passo {passo}:")
        for linha in estado:
            print(" ".join(" " if x == 0 else str(x) for x in linha))
        print("--------")

    print(f"nodes_expanded: {nos_expandidos}")
    print(f"running_time: {fim - inicio:.4f}")
    print(f"search depth: {search_depth}")
    print(f"max search depth: {max_search_depth}")
    print(f"fringe_size: {fringe_size}")
    print(f"max_fringe_size: {max_fringe_size}")

else:
    print("Não foi possível encontrar uma solução.")

tracemalloc.start()
a_estrela(estado_inicial)
memoria_usada = tracemalloc.get_traced_memory()[1]
tracemalloc.stop()
print(f"max_ram_usage: {memoria_usada}")

Estado inicial gerado:
[2, 6, 5]
[0, 1, 4]
[3, 8, 7]

cost_of_path: 25

Passo a passo da solução:

Passo 0:
2 6 5
  1 4
3 8 7
--------
Passo 1:
2 6 5
1   4
3 8 7
--------
Passo 2:
2 6 5
1 4  
3 8 7
--------
Passo 3:
2 6 5
1 4 7
3 8  
--------
Passo 4:
2 6 5
1 4 7
3   8
--------
Passo 5:
2 6 5
1   7
3 4 8
--------
Passo 6:
2 6 5
1 7  
3 4 8
--------
Passo 7:
2 6  
1 7 5
3 4 8
--------
Passo 8:
2   6
1 7 5
3 4 8
--------
Passo 9:
  2 6
1 7 5
3 4 8
--------
Passo 10:
1 2 6
  7 5
3 4 8
--------
Passo 11:
1 2 6
3 7 5
  4 8
--------
Passo 12:
1 2 6
3 7 5
4   8
--------
Passo 13:
1 2 6
3   5
4 7 8
--------
Passo 14:
1 2 6
  3 5
4 7 8
--------
Passo 15:
  2 6
1 3 5
4 7 8
--------
Passo 16:
2   6
1 3 5
4 7 8
--------
Passo 17:
2 3 6
1   5
4 7 8
--------
Passo 18:
2 3 6
1 5  
4 7 8
--------
Passo 19:
2 3  
1 5 6
4 7 8
--------
Passo 20:
2   3
1 5 6
4 7 8
--------
Passo 21:
  2 3
1 5 6
4 7 8
--------
Passo 22:
1 2 3
  5 6
4 7 8
--------
Passo 23:
1 2 3
4 5 6
  7 8
--------
Passo 24:
1 2 3
4 5 6
7

#Código completo Busca Gulosa

In [ ]:
import random
import time
import tracemalloc
import heapq
from collections import deque

def heuristica(estado):
    objetivo = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1), 0: (2, 2)
    }
    distancia = 0
    for i in range(3):
        for j in range(3):
            valor = estado[i][j]
            if valor != 0:
                objetivo_i, objetivo_j = objetivo[valor]
                distancia += abs(i - objetivo_i) + abs(j - objetivo_j)
    return distancia

def busca_gulosa(estado_inicial):
    objetivo = [[1,2,3],[4,5,6],[7,8,0]]
    objetivo_tuple = tuple(map(tuple, objetivo))

    fila = []
    heapq.heappush(fila, (heuristica(estado_inicial), estado_inicial))

    visitados = set()
    estado_anterior = {}
    profundidade = {tuple(map(tuple, estado_inicial)): 0}

    max_fringe_size = 1
    node_expanded = 0
    max_depth = 0

    while fila:
        _, estado_atual = heapq.heappop(fila)
        node_expanded += 1

        estado_tupla = tuple(map(tuple, estado_atual))

        if estado_tupla == objetivo_tuple:
            return caminho_total(estado_anterior, estado_atual), node_expanded, profundidade[estado_tupla], max_depth, len(fila), max_fringe_size

        if estado_tupla in visitados:
            continue

        visitados.add(estado_tupla)
        profundidade_atual = profundidade[estado_tupla]

        for vizinho in vizinhos(estado_atual):
            tupla_vizinho = tuple(map(tuple, vizinho))
            if tupla_vizinho not in visitados:
                estado_anterior[tupla_vizinho] = estado_tupla
                profundidade[tupla_vizinho] = profundidade_atual + 1
                prioridade = heuristica(vizinho)
                heapq.heappush(fila, (prioridade, vizinho))

                max_depth = max(max_depth, profundidade[tupla_vizinho])
                max_fringe_size = max(max_fringe_size, len(fila))

    return None, node_expanded, 0, max_depth, 0, max_fringe_size

def caminho_total(estado_anterior, estado_atual):
    caminho = [estado_atual]
    while tuple(map(tuple, estado_atual)) in estado_anterior:
        estado_atual = [list(row) for row in estado_anterior[tuple(map(tuple, estado_atual))]]
        caminho.append(estado_atual)
    return caminho[::-1]

def vizinhos(estado_atual):
    vizinho = []
    row, col = next((r, c) for r in range(3) for c in range(3) if estado_atual[r][c] == 0)
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        novo_r, novo_c = row + dr, col + dc
        if 0 <= novo_r < 3 and 0 <= novo_c < 3:
            novo_estado = [linha[:] for linha in estado_atual]
            novo_estado[row][col], novo_estado[novo_r][novo_c] = novo_estado[novo_r][novo_c], novo_estado[row][col]
            vizinho.append(novo_estado)
    return vizinho

def eh_soluvel(puzzle):
    flat_puzzle = [num for row in puzzle for num in row if num != 0]
    inversoes = 0
    for i in range(len(flat_puzzle)):
        for j in range(i + 1, len(flat_puzzle)):
            if flat_puzzle[i] > flat_puzzle[j]:
                inversoes += 1
    return inversoes % 2 == 0

def gerar_8puzzle():
    while True:
        elementos = list(range(9))
        random.shuffle(elementos)
        puzzle = [elementos[i:i+3] for i in range(0, 9, 3)]
        if eh_soluvel(puzzle):
            print("Estado inicial gerado:")
            for linha in puzzle:
                print(linha)
            return puzzle


estado_inicial = gerar_8puzzle()

inicio = time.time()
solucao, nos_expandidos, search_depth, max_search_depth, fringe_size, max_fringe_size = busca_gulosa(estado_inicial)
fim = time.time()

if solucao:
    print(f"\ncost_of_path: {len(solucao) - 1}")
    print("\nPasso a passo da solução:\n")
    for passo, estado in enumerate(solucao):
        print(f"Passo {passo}:")
        for linha in estado:
            print(" ".join(" " if x == 0 else str(x) for x in linha))
        print("--------")

    print(f"nodes_expanded: {nos_expandidos}")
    print(f"running_time: {fim - inicio:.4f}")
    print(f"search depth: {search_depth}")
    print(f"max search depth: {max_search_depth}")
    print(f"fringe_size: {fringe_size}")
    print(f"max_fringe_size: {max_fringe_size}")
else:
    print("Não foi possível encontrar uma solução.")


tracemalloc.start()
busca_gulosa(estado_inicial)
memoria_usada = tracemalloc.get_traced_memory()[1]
tracemalloc.stop()
print(f"max_ram_usage: {memoria_usada}")


Estado inicial gerado:
[4, 8, 0]
[3, 5, 1]
[7, 6, 2]

cost_of_path: 30

Passo a passo da solução:

Passo 0:
4 8  
3 5 1
7 6 2
--------
Passo 1:
4 8 1
3 5  
7 6 2
--------
Passo 2:
4 8 1
3 5 2
7 6  
--------
Passo 3:
4 8 1
3 5 2
7   6
--------
Passo 4:
4 8 1
3   2
7 5 6
--------
Passo 5:
4   1
3 8 2
7 5 6
--------
Passo 6:
4 1  
3 8 2
7 5 6
--------
Passo 7:
4 1 2
3 8  
7 5 6
--------
Passo 8:
4 1 2
3 8 6
7 5  
--------
Passo 9:
4 1 2
3 8 6
7   5
--------
Passo 10:
4 1 2
3   6
7 8 5
--------
Passo 11:
4 1 2
  3 6
7 8 5
--------
Passo 12:
  1 2
4 3 6
7 8 5
--------
Passo 13:
1   2
4 3 6
7 8 5
--------
Passo 14:
1 2  
4 3 6
7 8 5
--------
Passo 15:
1 2 6
4 3  
7 8 5
--------
Passo 16:
1 2 6
4 3 5
7 8  
--------
Passo 17:
1 2 6
4 3 5
7   8
--------
Passo 18:
1 2 6
4   5
7 3 8
--------
Passo 19:
1 2 6
4 5  
7 3 8
--------
Passo 20:
1 2  
4 5 6
7 3 8
--------
Passo 21:
1   2
4 5 6
7 3 8
--------
Passo 22:
1 5 2
4   6
7 3 8
--------
Passo 23:
1 5 2
4 3 6
7   8
--------
Passo 24:
1 5 2
4 3 6
7